In [ ]:
import pandas as pd

from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm
from normdata.utils import import_from_normdata

In [ ]:
df = gsheet_to_df("1aygcH8NTUuDVGSJVIVj1fyI4ATPv0x1Bt8vSLb76z3k").fillna("")

In [ ]:
BASE_URL = "https://schaubuehne.oeaw.ac.at"

In [ ]:
df = df[df["authorId"] != ""]
df["volume"] = df["volume"].astype(str).str.zfill(2)
df["half"] = df["half"].astype(str).str.zfill(2)
df["issue"] = df["issue"].astype(str).str.zfill(2)
df["pageNo"] = df["pageNo"].astype(str).str.zfill(4)

In [ ]:
df['author_url'] = BASE_URL + "/register.html?author=" + df['authorId'].astype(int).astype(str)

In [ ]:
df['issue_url'] = BASE_URL + "/issuenew-" + df["volume"] + "-" + df["half"] + "-" + df['issue'] + ".html#page=n" + df["pageNo"]

In [ ]:
df["label"] = "Die Schaubühne, " + df["displayText"] + ", S. " + df["pageNo"].astype(int).astype(str)

In [ ]:
df.to_csv("hansi.csv", index=False)

In [ ]:
col, _ = Collection.objects.get_or_create(name="Die Schaubühne")
domain = "schaubuehne"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = """
„Die Schaubühne. Herausgeber: Siegfried Jacobsohn, Berlin 1905–1918“. Digitale Edition. Hrsg. v. Imelda Rohrbacher. Wien: Österreichische Akademie der Wissenschaften (ÖAW) 2024, <a href="https://schaubuehne.oeaw.ac.at">https://schaubuehne.oeaw.ac.at</a>
"""
col.collection_type = col_type
col.save()
person_work = PersonWorkRelation.objects.get(id=1049)

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    domain_uri = row["issue_url"]
    
    try:
        uri = Uri.objects.get(uri=domain_uri)
        entity = uri.entity
        entity = Work.objects.get(id=entity.id)
    except ObjectDoesNotExist:
        item = {
            "name": row["title"],
            "start_date_written": row["date"]
        }
        entity = Work.objects.create(**item)
        entity.collection.add(col)
        uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
        uri.entity = entity
        uri.save()
    try:
        author_uri = Uri.objects.get(uri=row["author_url"])
    except ObjectDoesNotExist:
        continue
    author_ent = author_uri.entity
    author = Person.objects.get(id=author_ent.id)
    geschaffen, _ = PersonWork.objects.get_or_create(
        related_person=author,
        related_work=entity,
        start_date_written=row["date"],
        relation_type=person_work
    )

In [ ]:
# items = Work.objects.filter(collection__name="Die Schaubühne")
# items.delete()

In [ ]:
# items = Uri.objects.filter(uri__icontains="issuenew")
# items.delete()

In [ ]:
# ToDo: Labels